In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [29]:
def load_distance_time_matrix(path, N):
    data= pd.read_csv(path)
    distance = np.zeros((N,N))
    time = np.zeros((N,N))
    
    for i in range(len(data)):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen-1, destino-1] = float(data.iloc[i, 2])
        time[origen-1, destino-1] = float(data.iloc[i, 3])
    return distance, time

def load_vehicles(path):
    data= pd.read_csv(path)
    N = len(data)
    vehicles = {}
    for i in range(N):
        id = int(data.iloc[i, 0])
        capacity = int(data.iloc[i, 1])
        ran = float(data.iloc[i, 2])
        vehicles[id] = (capacity, ran)
    
    return vehicles

def load_demand(clientsPath):
    data= pd.read_csv(clientsPath)
    N = len(data)
    demand_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        demand = float(data.iloc[i, 2])
        demand_dic[id] = demand
    
    return demand_dic


def load_capacity(depositPath):
    data= pd.read_csv(depositPath)
    N = len(data)
    capacity_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        capacity = float(data.iloc[i, 4])
        capacity_dic[id] = capacity
    
    return capacity_dic

def load_distance_time_dic(path, N):
    data= pd.read_csv(path)
    distance = {}
    time = {}
    
    for i in range(N):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen, destino] = float(data.iloc[i, 2])
        time[origen, destino] = float(data.iloc[i, 3])
    
    return distance, time


In [30]:
#Datos artificiales

np.random.seed(42)
n = 25  # 24 clientes + 1 depósito

# Crear matriz de distancias aleatorias simétrica

#dist_matrix_np = np.random.uniform(2, 25, size=(n, n))
#dist_matrix_np = (dist_matrix_np + dist_matrix_np.T) / 2

dist_matrix_np, time_matrix = load_distance_time_matrix('..\Datos\Caso_Base\casoBase.csv', 25)
# Convertir a diccionario {(i,j): distancia}
distancia = {(i, j): dist_matrix_np[i][j] for i in range(n) for j in range(n)}

"""
vehiculos = {
  1: (130, 170),
  2: (140, 200),
  3: (120, 180),
  4: (100, 90),
  5: (70, 100),
  6: (55, 170),
  7: (110, 150),
  8: (114, 140)
}

demanda = {
  0:0, 1: 13, 2: 15, 3: 12, 4: 15, 5: 20, 6: 17, 7: 17, 8: 20, 9: 20, 10: 15,
  11: 17, 12: 12, 13: 21, 14: 15, 15: 17, 16: 10, 17: 25, 18: 12,
  19: 11, 20: 15, 21: 14, 22: 18, 23: 15, 24: 11
}
"""

vehiculos = load_vehicles('../Datos/Caso_Base/vehicles.csv')
demanda = load_demand('..\Datos\Caso_Base\clients.csv')

print(vehiculos)
print(demanda)



{1: (130, 170.0), 2: (140, 200.0), 3: (120, 180.0), 4: (100, 90.0), 5: (70, 100.0), 6: (55, 170.0), 7: (110, 150.0), 8: (114, 140.0)}
{2: 13.0, 3: 15.0, 4: 12.0, 5: 15.0, 6: 20.0, 7: 17.0, 8: 17.0, 9: 20.0, 10: 20.0, 11: 15.0, 12: 17.0, 13: 12.0, 14: 21.0, 15: 15.0, 16: 17.0, 17: 10.0, 18: 25.0, 19: 12.0, 20: 11.0, 21: 15.0, 22: 14.0, 23: 18.0, 24: 15.0, 25: 11.0}


In [32]:
model = ConcreteModel()

# Conjuntos e índices

model.N = Set(initialize=range(0, n+1))  # Todos los nodos (0 = depósito)
model.C = Set(initialize=demanda.keys())  # Clientes
model.V = Set(initialize=vehiculos.keys())  # Vehículos
model.D = Set(initialize=[0])  # Nodo depósito


# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)  # Ruta (i->j) por vehículo v
model.u = Var(model.N, model.V, domain=NonNegativeReals)  # Carga acumulada en nodo i por vehículo v

# Parámetros

model.dist = Param(model.N, model.N, initialize=distancia, default=0)
model.demand = Param(model.C, initialize=demanda)
model.cap = Param(model.V, initialize={v: vehiculos[v][0] for v in vehiculos})
model.range = Param(model.V, initialize={v: vehiculos[v][1] for v in vehiculos})



# Función Objetivo

def obj_rule(m):
    return sum(m.dist[i, j] * m.x[i, j, v] for i in m.N for j in m.N if i != j for v in m.V)
model.obj = Objective(rule=obj_rule, sense=minimize)


# Restricciones

# Cada cliente es visitado exactamente una vez
model.visit_once = ConstraintList()
for c in model.C:
    model.visit_once.add(
        sum(model.x[i, c, v] for i in model.N if i != c for v in model.V) == 1
    )

# Flujo conservado por vehículo en cada cliente
model.flow_conservation = ConstraintList()
for v in model.V:
    for c in model.C:
        model.flow_conservation.add(
            sum(model.x[i, c, v] for i in model.N if i != c) ==
            sum(model.x[c, j, v] for j in model.N if j != c)
        )

# Salida del depósito por vehículo es máximo 1
model.depot_departure = ConstraintList()
for v in model.V:
    model.depot_departure.add(
        sum(model.x[0, j, v] for j in model.N if j != 0) <= 1
    )

# Capacidad del vehículo no debe excederse
model.capacity_constraint = ConstraintList()
for v in model.V:
    model.capacity_constraint.add(
        sum(model.demand[i] * model.x[i, j, v] for i in model.C for j in model.N if i != j) <= model.cap[v]
    )
# Valor inicial de carga en el depósito es 0
model.initial_load = ConstraintList()
for v in model.V:
    model.initial_load.add(model.u[0, v] == 0)

# La carga nunca puede superar la capacidad del vehículo
model.max_capacity = ConstraintList()
for v in model.V:
    for i in model.N:
        model.max_capacity.add(model.u[i, v] <= model.cap[v])

# Rango del vehículo no debe excederse
model.range_constraint = ConstraintList()
for v in model.V:
    model.range_constraint.add(
        sum(model.dist[i, j] * model.x[i, j, v] for i in model.N for j in model.N if i != j) <= model.range[v]
    )

# No permitir ciclos triviales (i -> i)
model.no_loops = ConstraintList()
for v in model.V:
    for i in model.N:
        model.no_loops.add(model.x[i, i, v] == 0)


opt = SolverFactory('glpk')
results = opt.solve(model, tee=True)
print(results.solver.status)
print(results.solver.termination_condition)

print("Valor de la función objetivo:", model.obj())




GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\laura\AppData\Local\Temp\tmpqrsmhc8o.glpk.raw --wglp C:\Users\laura\AppData\Local\Temp\tmpsqz6o6a7.glpk.glp
 --cpxlp C:\Users\laura\AppData\Local\Temp\tmp01z9bv3r.pyomo.lp
Reading problem data from 'C:\Users\laura\AppData\Local\Temp\tmp01z9bv3r.pyomo.lp'...
C:\Users\laura\AppData\Local\Temp\tmp01z9bv3r.pyomo.lp:37042: warning: lower bound of variable 'x5867' redefined
C:\Users\laura\AppData\Local\Temp\tmp01z9bv3r.pyomo.lp:37042: warning: upper bound of variable 'x5867' redefined
664 rows, 5616 columns, 24624 non-zeros
5408 integer variables, all of which are binary
42450 lines were read
Writing problem data to 'C:\Users\laura\AppData\Local\Temp\tmpsqz6o6a7.glpk.glp'...
36171 lines were written
GLPK Integer Optimizer 5.0
664 rows, 5616 columns, 24624 non-zeros
5408 integer variables, all of which are binary
Preprocessing...
232 rows, 400 columns, 1536 non-zeros
400 integer variables, all of whic

ValueError: No value for uninitialized NumericValue object x[0,1,1]